In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch
from torch import from_numpy, cuda
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
from torchvision import transforms
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR, SequentialLR, LinearLR
from PIL import Image
from torch.cuda.amp import GradScaler, autocast

In [2]:
device = torch.device("cuda" if cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
with open("food-101/meta/classes.txt", 'r') as f:
    classes = f.read().splitlines()
len(classes)

101

In [4]:
class Label_encoder:
    def __init__(self, labels):
        self.labels = {label: idx for idx, label in enumerate(labels)}
    def get_label(self, idx):
        return list(self.labels.keys())[idx]
    def get_idx(self, label):
        return self.labels.get(label)

encoder = Label_encoder(classes)

In [5]:
# Printing each class with its corresponding index
for i in range(101):
    print(encoder.get_label(i), encoder.get_idx(encoder.get_label(i)))

apple_pie 0
baby_back_ribs 1
baklava 2
beef_carpaccio 3
beef_tartare 4
beet_salad 5
beignets 6
bibimbap 7
bread_pudding 8
breakfast_burrito 9
bruschetta 10
caesar_salad 11
cannoli 12
caprese_salad 13
carrot_cake 14
ceviche 15
cheesecake 16
cheese_plate 17
chicken_curry 18
chicken_quesadilla 19
chicken_wings 20
chocolate_cake 21
chocolate_mousse 22
churros 23
clam_chowder 24
club_sandwich 25
crab_cakes 26
creme_brulee 27
croque_madame 28
cup_cakes 29
deviled_eggs 30
donuts 31
dumplings 32
edamame 33
eggs_benedict 34
escargots 35
falafel 36
filet_mignon 37
fish_and_chips 38
foie_gras 39
french_fries 40
french_onion_soup 41
french_toast 42
fried_calamari 43
fried_rice 44
frozen_yogurt 45
garlic_bread 46
gnocchi 47
greek_salad 48
grilled_cheese_sandwich 49
grilled_salmon 50
guacamole 51
gyoza 52
hamburger 53
hot_and_sour_soup 54
hot_dog 55
huevos_rancheros 56
hummus 57
ice_cream 58
lasagna 59
lobster_bisque 60
lobster_roll_sandwich 61
macaroni_and_cheese 62
macarons 63
miso_soup 64
mussels

In [6]:
class Food101(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.path.iloc[idx]
        image = Image.open(img_name)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        label = encoder.get_idx(self.dataframe.label.iloc[idx])

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
def prep_df(path: str) -> pd.DataFrame:
    with open(path, 'r') as f:
        names = f.read().splitlines()
    img_path = "food-101/images/"
    full_path = [img_path + img + ".jpg" for img in names]
    imgs = []
    for img in names:
        img = img.split('/')
        imgs.append(img)
    imgs = np.array(imgs)
    
    df = pd.DataFrame(imgs[:, 0], imgs[:,1], columns=['label'])      
    df['path'] = full_path
    df = shuffle(df)
    return df

train_imgs = prep_df('food-101/meta/train.txt')
test_imgs = prep_df('food-101/meta/test.txt')

train_imgs.groupby(["label"]).count().reset_index()

,label,path
0,apple_pie,750
1,baby_back_ribs,750
2,baklava,750
3,beef_carpaccio,750
4,beef_tartare,750
...,...,...
96,tacos,750
97,takoyaki,750
98,tiramisu,750
99,tuna_tartare,750


In [8]:
train_imgs.head()

,label,path
274977,beignets,food-101/images/beignets/274977.jpg
3330276,fish_and_chips,food-101/images/fish_and_chips/3330276.jpg
1691717,beet_salad,food-101/images/beet_salad/1691717.jpg
1992603,poutine,food-101/images/poutine/1992603.jpg
1224294,guacamole,food-101/images/guacamole/1224294.jpg


In [9]:
class CutOut(object):
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        h, w = img.size(1), img.size(2)
        mask = np.ones((h, w), np.float32)

        for _ in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)
            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)
            mask[y1: y2, x1: x2] = 0.

        mask = from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask
        return img

In [10]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    CutOut(n_holes=1, length=16)
])


transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [11]:
trainset = Food101(train_imgs, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = Food101(test_imgs, transform=transform_test)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [12]:
class FoodCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),  # Added BatchNorm
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 4
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7)),  # Adaptive pooling
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 7 * 7, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),  # BatchNorm in classifier
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = FoodCNN(len(classes)).to(device)

In [13]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=5e-4)

warmup_epochs = 3
total_epochs = 11

scheduler = SequentialLR(
    optimizer,
    schedulers=[
        LinearLR(optimizer, start_factor=0.1, total_iters=warmup_epochs),
        CosineAnnealingLR(optimizer, T_max=total_epochs - warmup_epochs)
    ],
    milestones=[warmup_epochs]
)


In [14]:
def train(epoch):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    loop = tqdm(trainloader, desc=f"Training Epoch {epoch + 1}", leave=False)

    for inputs, targets in loop:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
    
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()

        loop.set_postfix(loss=loss.item())

    train_accuracy = 100. * correct_train / total_train
    avg_loss = running_loss / len(trainloader)
    print(f"Epoch {epoch+1}: Train Loss = {avg_loss:.4f}, Accuracy = {train_accuracy:.2f}%")

# --- Тестирование ---
def test(epoch):
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, targets in tqdm(testloader, desc=f"Testing Epoch {epoch + 1}", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_test += targets.size(0)
            correct_test += predicted.eq(targets).sum().item()

    test_accuracy = 100. * correct_test / total_test
    print(f"Test Accuracy after Epoch {epoch + 1}: {test_accuracy:.2f}%")
    return test_accuracy

In [15]:
best_acc = 0.0
print("Starting training...\n")

for epoch in range(total_epochs):
    train(epoch)
    acc = test(epoch)
    scheduler.step()

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "food101_best.pt")
        print(f"✅ Best model saved with accuracy: {best_acc:.2f}%")
print("\n🎉 Training completed.")

Starting training...



Epoch 1: Train Loss = 4.5844, Accuracy = 3.45%


Test Accuracy after Epoch 1: 10.03%
✅ Best model saved with accuracy: 10.03%


Epoch 2: Train Loss = 4.4390, Accuracy = 5.28%


Test Accuracy after Epoch 2: 12.73%
✅ Best model saved with accuracy: 12.73%


...

Test Accuracy after Epoch 9: 32.59%
✅ Best model saved with accuracy: 32.59%


Epoch 10: Train Loss = 3.6947, Accuracy = 20.32%


Test Accuracy after Epoch 10: 34.15%
✅ Best model saved with accuracy: 34.15%


Epoch 11: Train Loss = 3.6689, Accuracy = 21.01%


Test Accuracy after Epoch 11: 35.06%
✅ Best model saved with accuracy: 35.06%

🎉 Training completed.


In [18]:
checkpoint = {
    'epoch': total_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'rng_state': torch.get_rng_state()
}

torch.save(checkpoint, f'model_epoch_{total_epochs}.pth')
torch.save(checkpoint, f'model_epoch_{total_epochs}.tar')

In [28]:
for epoch in range(11, 28):
    train(epoch)
    acc = test(epoch)
    scheduler.step()

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "food101_best.pt")
        print(f"✅ Best model saved with accuracy: {best_acc:.2f}%")
    if epoch % 3 == 0:
        checkpoint = {
            'epoch': total_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'rng_state': torch.get_rng_state()
        }
    
        torch.save(checkpoint, f'model_epoch_{total_epochs}.pth')
        torch.save(checkpoint, f'model_epoch_{total_epochs}.tar')
print("\n🎉 Training completed.")

Epoch 12: Train Loss = 3.6667, Accuracy = 21.05%


Test Accuracy after Epoch 12: 35.05%


Epoch 13: Train Loss = 3.6583, Accuracy = 21.22%


Test Accuracy after Epoch 13: 35.26%
✅ Best model saved with accuracy: 35.26%


Epoch 14: Train Loss = 3.6627, Accuracy = 21.08%


...

Test Accuracy after Epoch 26: 44.70%
✅ Best model saved with accuracy: 44.70%


Epoch 27: Train Loss = 3.3751, Accuracy = 28.47%


Test Accuracy after Epoch 27: 45.10%
✅ Best model saved with accuracy: 45.10%


Epoch 28: Train Loss = 3.3659, Accuracy = 28.70%


Test Accuracy after Epoch 28: 44.84%

🎉 Training completed.


In [27]:
for epoch in range(11, 28):
    if epoch % 3 == 0:
        print('__', epoch)
    print(epoch)

11
__ 12
12
13
14
__ 15
15
16
17
__ 18
18
19
20
__ 21
21
22
23
__ 24
24
25
26
__ 27
27
